In [2]:
import numpy as np
from sympy import symbols, cos, sin, Eq, solve
import os 
import json
def extract_arrays(file_path, line_number=19):
    def parse_array(line, marker):
        # Locate the start position of the marker
        start_pos = line.find(marker)
        if start_pos == -1:
            return []

        # Locate the content after the marker and the closing bracket
        start_pos += len(marker)
        end_pos = line.find(']', start_pos) + 1

        # Extract the array string
        array_str = line[start_pos:end_pos].strip().lstrip('[').rstrip(']')

        # Convert to floats
        array = [float(item.strip()) for item in array_str.split(',')]

        return array

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extracting the line
    line_extracted = lines[line_number].strip()

    # Parsing v and P arrays
    v = np.array(parse_array(line_extracted, 'v='))
    P = np.array(parse_array(line_extracted, 'P='))

    return v, P

isocenter = np.array([ 0.61292762, -8.52901228,  0.77052719])

dataset_folder = '/home/pablo/HeadPlans/HN-CHUM-018/dataset12'

# Reference array without deviations
v, P= extract_arrays(os.path.join(dataset_folder, 'sobp0/field0/fred.inp'), line_number=17)  # 18 for older datasets
v_pb2, P_pb2 = extract_arrays(os.path.join(dataset_folder, 'sobp0/field0/fred.inp'), line_number=18) # 19 for older datasets

p = -isocenter + P + 8 * v  # multiplying by 8 because it is how much we displace the source from the target in gen_fred_head_dataset
p_pb2 = -isocenter + P_pb2 + 8 * v_pb2  # in previous datasets, its 10

dict_deviations = {}

num_sobps = 200
print(f"Number of SOBPs: {num_sobps}")
for i in range(num_sobps):
    print(f"Processing SOBP {i}")
    sobp_num = f"sobp{i}"
    # Array with deviations
    v, P= extract_arrays(os.path.join(dataset_folder, f'{sobp_num}/field0/fred.inp'), line_number=17) # 18 for older datasets
    v_pb2, P_pb2 = extract_arrays(os.path.join(dataset_folder, f'{sobp_num}/field0/fred.inp'), line_number=18) # 19 for older datasets
                
    p_prime = -isocenter + P + 8 * v
    p_prime_pb2 = -isocenter + P_pb2 + 8 * v_pb2

    # z, x
    z, x, y = symbols('z x y')
    eq1 = Eq(p[0] * cos(z) - sin(z) * p[2], -x + p_prime[0])
    eq2 = Eq(p[0] * sin(z) + cos(z) * p[2], -y + p_prime[2])
    eq3 = Eq(p_pb2[0] * cos(z) - sin(z) * p_pb2[2], -x + p_prime_pb2[0])

    solution = solve((eq1, eq2, eq3), (z, x, y))
    # Choose only the solution where the three values are below 0.5 in absolute value
    for sol in solution:
        for i in range(3):
            sol = list(sol)
            sol[i] = float(sol[i])
        if (abs(sol[1]) < 0.5 and abs(sol[2]) < 0.5 and abs(sol[0]) < 5.0 * np.pi / 180):
            # permute solution to [1, 2, 0]    
            final_sol = [sol[1], sol[2], sol[0] * 180 / np.pi]
            

    dict_deviations[sobp_num] = final_sol
    
deviations_path = os.path.join(dataset_folder, "deviations_from_inp.json")

# print(dict_deviations)
with open(deviations_path, 'w') as jsonfile:
    json.dump(dict_deviations, jsonfile)

Number of SOBPs: 200
Processing SOBP 0
Processing SOBP 1
Processing SOBP 2
Processing SOBP 3
Processing SOBP 4
Processing SOBP 5
Processing SOBP 6
Processing SOBP 7
Processing SOBP 8
Processing SOBP 9
Processing SOBP 10
Processing SOBP 11
Processing SOBP 12
Processing SOBP 13
Processing SOBP 14
Processing SOBP 15
Processing SOBP 16
Processing SOBP 17
Processing SOBP 18
Processing SOBP 19
Processing SOBP 20
Processing SOBP 21
Processing SOBP 22
Processing SOBP 23
Processing SOBP 24
Processing SOBP 25
Processing SOBP 26
Processing SOBP 27
Processing SOBP 28
Processing SOBP 29
Processing SOBP 30
Processing SOBP 31
Processing SOBP 32
Processing SOBP 33
Processing SOBP 34
Processing SOBP 35
Processing SOBP 36
Processing SOBP 37
Processing SOBP 38
Processing SOBP 39
Processing SOBP 40
Processing SOBP 41
Processing SOBP 42
Processing SOBP 43
Processing SOBP 44
Processing SOBP 45
Processing SOBP 46
Processing SOBP 47
Processing SOBP 48
Processing SOBP 49
Processing SOBP 50
Processing SOBP 51
P